In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("us.env"))  # read local .env file

In [2]:
from pathlib import Path
from tqdm import tqdm
import sys

sys.path.append(Path("..").resolve().as_posix())

In [3]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings

openai_api_version = "2023-07-01-preview"

gpt4 = AzureOpenAI(
    azure_deployment="gpt-4-1106",
    openai_api_version=openai_api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    deployment_name="text-embedding-ada-002",
    api_version=openai_api_version,
    embed_batch_size=100,
)

Settings.llm = gpt4
Settings.embed_model = embed_model

In [4]:
# Load query engine

from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.query_engine import RetrieverQueryEngine

persist_path = Path("index_base_storage").resolve()

storage_context = StorageContext.from_defaults(persist_dir=persist_path.as_posix())
index = load_index_from_storage(storage_context)


# configure retriever
retriever = index.as_retriever(
    similarity_top_k=10,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
)

query_engine = index.as_query_engine()

In [5]:
from llama_index.core import PromptTemplate


new_summary_tmpl_str = (
    """As an enthusiastic EdgeDB expert keen to assist, respond to queries in markdown, referencing the given EdgeDB sections and previous history context.
    If a "Learn more" link appears in the context, verify if it starts with "https://www.edgedb.com/".
    If so, append it to the answer, otherwise exclude it.
    Ensure to utilize the "new syntax" in any ```sdl blocks within the answer, replacing old syntax.
    The new syntax uses "->" over ":", and omits "property" and "link" for non-computed properties/links. See below:

    Old:
    ```sdl
    type Movie {{
    required property title -> str;
    multi link actors -> Person;
    }}
    ```
    New:
    ``sdl
    type Movie {{
    required title: str;
    multi actors: Person;
    }}
    ```
    If unable to help based on documentation, respond with: "Sorry, I don't know how to help with that."
    EdgeDB sections: ***
    {context_str}
    ***

    Question: ***
    {query_str}
    ***

    Answer in markdown (including related code snippets if available).

    After the answer, add a short summary of the question that can be used as a chat title.
    Prefix it with: "????". The summary should be no more than 35 characters.
    """
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

query_engine.update_prompts({"response_synthesizer:text_qa_template": new_summary_tmpl})

In [6]:
easy_query_path = Path("benchmarks/tutorial_benchmark.txt").resolve()
# hard_query_path = Path("benchmarks/user_benchmark.txt").resolve()
hard_query_path = Path("benchmarks/new_hard.txt").resolve()

In [7]:
from IPython.core.display import Markdown


# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [8]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

As an enthusiastic EdgeDB expert keen to assist, respond to queries in markdown, referencing the given EdgeDB sections and previous history context.
    If a "Learn more" link appears in the context, verify if it starts with "https://www.edgedb.com/".
    If so, append it to the answer, otherwise exclude it.
    Ensure to utilize the "new syntax" in any ```sdl blocks within the answer, replacing old syntax.
    The new syntax uses "->" over ":", and omits "property" and "link" for non-computed properties/links. See below:

    Old:
    ```sdl
    type Movie {{
    required property title -> str;
    multi link actors -> Person;
    }}
    ```
    New:
    ``sdl
    type Movie {{
    required title: str;
    multi actors: Person;
    }}
    ```
    If unable to help based on documentation, respond with: "Sorry, I don't know how to help with that."
    EdgeDB sections: ***
    {context_str}
    ***

    Question: ***
    {query_str}
    ***

    Answer in markdown (including related code

<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [9]:
with easy_query_path.open("r") as f:
    easy_queries = [query.strip() for query in f.readlines()]

with hard_query_path.open("r") as f:
    hard_queries = f.read().split(">>>SEPARATOR<<<")

In [10]:
from src.eval import PydanticResponse, run_queries

In [11]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
responses = run_queries(query_engine, hard_queries, Path(f"eval_old_hard_{timestamp}.jsonl").resolve())

100%|██████████| 183/183 [1:13:35<00:00, 24.13s/it]
